# RAG & DLT with data from Notion API

Repeat the same process for a third page titled **"Homework: Employee handbook"** (hidden from public view, but accessible via API key):

- Modify the REST API source to extract only this page.
- Write the output into a separate table called "homework".
- Remember to update the table name in all cells where you connect to a lancedb table.

## Part 1: Create a Notion -> LanceDB pipeline using dlt

In [3]:
import os

from datetime import datetime, timezone
from dotenv import load_dotenv, find_dotenv


# load ENV vars
load_dotenv(find_dotenv(".env"))

os.environ["SOURCES__REST_API__NOTION__API_KEY"] = os.environ.get("NOTION__API_KEY")
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"
os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"

In [4]:
import dlt
from rest_api import RESTAPIConfig, rest_api_source

from dlt.sources.helpers.rest_client.paginators import BasePaginator, JSONResponsePaginator
from dlt.sources.helpers.requests import Response, Request

from dlt.destinations.adapters import lancedb_adapter

In [14]:
class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response: Response) -> None:
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}

        # Add the cursor to the request body
        request.json["start_cursor"] = self.cursor


@dlt.resource(name="homework")
def rest_api_notion_resource():
    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": dlt.secrets["sources.rest_api.notion.api_key"]
            },
            "headers":{
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "homework",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }

    yield from rest_api_source(notion_config, name="homework")


def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc)
    }


@dlt.resource(
    name="homework",
    write_disposition="merge",
    primary_key="block_id",
    columns={"last_edited_time":{"dedup_sort":"desc"}}
    )
def rest_api_notion_incremental(
    last_edited_time = dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z",primary_key=("block_id"))
):
    last_value = last_edited_time.last_value
    print(last_value)

    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not(len(block["content"])):
            continue
        yield block


def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="company_policies",
        destination="lancedb",
        dataset_name="notion_pages",
        # full_refresh=True
    )

    load_info = pipeline.run(
        lancedb_adapter(
            rest_api_notion_incremental,
            embed="content"
        ),
        table_name="homework",
        write_disposition="merge"
    )
    print(load_info)


load_notion()

2024-07-05T23:33:00.000Z
Pipeline company_policies load step completed in ---
0 load package(s) were loaded to destination LanceDB and into dataset None
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x17810fb50> location to store data


### Q1. Rows in LanceDB

How many rows does the lancedb table "notion_pages__homework" have?

In [19]:
import lancedb


db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___homework")

df_homework = dbtable.to_pandas()
print("Shape:", df_homework.shape)

Shape: (17, 9)


### Q2. Running the Pipeline: Last edited time

In the demo, we created an incremental dlt resource rest_api_notion_incremental that keeps track of last_edited_time. What value does it store after you've run your pipeline once? (Hint: you will be able to get this value by performing some aggregation function on the column last_edited_time of the table)

In [13]:
df_homework.last_edited_time.max()

Timestamp('2024-07-05 23:33:00+0000', tz='UTC')

## Part 2: Create a RAG bot using Ollama

In [15]:
import ollama

In [16]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):

    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

In [17]:
def rag(question: str):
    
    # Connect to the lancedb table
    db = lancedb.connect(".lancedb")
    dbtable = db.open_table("notion_pages___homework")
    
    # A system prompt telling ollama to accept input in the form of "Question: ... ; Context: ..."
    messages = [
      {
          "role": "system",
          "content": (
              "You are a helpful assistant that helps users understand policies inside a company's employee handbook."
              " The user will first ask you a question and then provide you relevant paragraphs from the handbook as context."
              " Please answer the question based on the provided context."
              " For any details missing in the paragraph, encourage the employee to contact the HR for that information."
              " Please keep the responses conversational."
          )
      }
    ]
    
    # Retrieve the relevant paragraphs on the question
    context = retrieve_context_from_lancedb(dbtable, question, top_k=2)
    
    # Create a user prompt using the question and retrieved context
    messages.append(
        {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
    )
    
    # Get the response from the LLM
    response = ollama.chat(
        model="llama3",
        messages=messages
    )
    response_content = response['message']['content']
    print(f"Assistant: {response_content}")
    
    # Add the response into the context window
    messages.append(
        {"role": "assistant", "content":response_content}
    )
    print("-" * 20)

### Q3. Ask the Assistant

Find out with the help of the AI assistant: how many PTO days are the employees entitled to in a year?

In [18]:
rag(question="How many PTO days are the employees entitled to in a year?")

Assistant: Based on the provided context, it seems like employees are entitled to 30 days of Paid Time Off (PTO) per year. You earn 2.5 days of PTO per month, which accrues from your first day with the company. If you're wondering about any additional details, such as how many PTO days you'll have after your first year or what happens to accrued leave when you leave the company, I'd encourage you to reach out to HR for more information. They should be able to provide you with the specific answers you're looking for!
--------------------


<hr>